In [1]:
from data_processing import import_and_clean_train, one_hot_encoding, label_encoding


train_x_le, test_x_le, train_y_le, test_y_le, label_dict = import_and_clean_train('train', label_encoding)
train_x_ohe, test_x_ohe, train_y_ohe, test_y_ohe = import_and_clean_train('train', one_hot_encoding)


{'Electrical': 1.0}
{'Electrical': 1.0}


In [2]:
# Imports
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import seaborn as sns



# Make a model with onehot
LinearModel_ohe = LinearRegression()
# Fit the data
LinearModel_ohe.fit(train_x_ohe, train_y_ohe)
# Predict train
train_prediction_reg_ohe = LinearModel_ohe.predict(train_x_ohe)
# Predict test
y_prediction_reg_ohe = LinearModel_ohe.predict(test_x_ohe)

# Make a model with label
LinearModel_le = LinearRegression()
# Fit the data
LinearModel_le.fit(train_x_le, train_y_le)
# Predict train
train_prediction_reg_le = LinearModel_le.predict(train_x_le)
# Predict test
y_prediction_reg_le = LinearModel_le.predict(test_x_le)



# Creating forests with label
regr_le = RandomForestRegressor(n_jobs=-1)
regr_le.fit(train_x_le, train_y_le)
# Getting prediction and calculating accuracy
y_prediction_forest_le = regr_le.predict(test_x_le)
train_prediction_forest_le = regr_le.predict(train_x_le)

# Creating forests with one_hot
regr_ohe = RandomForestRegressor(n_jobs=-1)
regr_ohe.fit(train_x_ohe, train_y_ohe)
# Getting prediction and calculating accuracy
y_prediction_forest_ohe = regr_ohe.predict(test_x_ohe)
train_prediction_forest_ohe = regr_ohe.predict(train_x_ohe)




# Calculate average deviation
def dev(y, y_hat):
    tot = 0
    for i in range(len(y)):
        tot += abs(y[i]-y_hat[i])
    return tot / len(y)

# Regression ohe
print('Regression ohe')
print('test', dev(test_y_le, y_prediction_reg_ohe))
print('train', dev(train_y_le, train_prediction_reg_ohe))
print('\n')

# Regression le
print('Regression le')
print('test', dev(test_y_le, y_prediction_reg_le))
print('train', dev(train_y_le, train_prediction_reg_le))
print('\n')


# Forest ohe
print('Forest ohe')
print('test', dev(test_y_le, y_prediction_forest_ohe))
print('train', dev(train_y_le, train_prediction_forest_ohe), '\n')

# Forest le
print('Forest le')
print('test', dev(test_y_le, y_prediction_forest_le))
print('train', dev(train_y_le, train_prediction_forest_le))

Regression ohe
test 17490.33246539653
train 13262.67416835545


Regression le
test 18283.651801462194
train 20337.352455081633


Forest ohe
test 16095.337716894988
train 6834.471878669269 

Forest le
test 15785.628767123293
train 6667.156428571429


In [ ]:
random_grid = {'n_estimators': [700, 900, 1100, 1300], 'max_depth': [15, 20, 25, 30, 35], 'max_features': [0.2, 0.3, 0.4, 0.5, 0.6, 'sqrt', 'log2'], 'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3, 4], 'bootstrap': [False]}

random_search = RandomizedSearchCV(estimator = regr_le, param_distributions = random_grid, n_jobs=-1, cv=20)
random_search.fit(train_x_le, train_y_le)
print(random_search.best_params_)

In [ ]:
search_grid = {'n_estimators': [1100, 1300], 'max_depth': [25, 30, 35], 'max_features': [0.3, 0.4, 0.5], 'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3], 'bootstrap': [False]}

grid_search = GridSearchCV(estimator= regr_le, param_grid=search_grid, n_jobs=-1)
grid_search.fit(train_x_le, train_y_le)
print(grid_search.best_params_)

In [3]:
y_pred = 0
train_pred = 0

for i in range(20):
    # Creating forests with label and the best parameters
    regr_le_random = RandomForestRegressor(max_depth= 35, max_features= 0.3, min_samples_leaf= 1, n_estimators= 1100, min_samples_split=2, bootstrap=False, n_jobs=-1)
    regr_le_random.fit(train_x_le, train_y_le)
    # Getting prediction and calculating accuracy
    y_prediction_forest_le_random = regr_le_random.predict(test_x_le)
    train_prediction_forest_le_random = regr_le_random.predict(train_x_le)
    
    y_pred += dev(test_y_le, y_prediction_forest_le_random) / 20
    train_pred += dev(train_y_le, train_prediction_forest_le_random) / 20

# Forest le grid
print('Forest le random')
print('test', y_pred)
print('train', train_pred, '\n')

Forest le random
test 14244.027914694894
train 0.0013787582280735554 



In [4]:
y_pred = 0
train_pred = 0

for i in range(20):
    # Creating forests with label and the best parameters
    regr_le_random = RandomForestRegressor(max_depth= 20, max_features= 0.3, min_samples_leaf= 1, n_estimators= 1100, min_samples_split=2, bootstrap=False, n_jobs=-1)
    regr_le_random.fit(train_x_ohe, train_y_ohe)
    # Getting prediction and calculating accuracy
    y_prediction_forest_le_random = regr_le_random.predict(test_x_ohe)
    train_prediction_forest_le_random = regr_le_random.predict(train_x_ohe)
    
    y_pred += dev(test_y_ohe, y_prediction_forest_le_random) / 20
    train_pred += dev(train_y_ohe, train_prediction_forest_le_random) / 20

# Forest le grid
print('Forest ohe random')
print('test', y_pred)
print('train', train_pred, '\n')

Forest ohe random
test 14203.387599299522
train 20.882596107664526 

